In [29]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from catboost import Pool
import numpy as np


In [32]:
raw_data_actions = pd.read_csv('../opponents_move/data/users_hands_all_states_df.csv')

Columns to drop

In [39]:
for col in raw_data_actions.columns:
    if len(set(raw_data_actions[col])) == 1:
        raw_data_actions.drop(col, axis=1, inplace=True)
    else:
        print(col)

unique_id
user_step_num
hand_id
user_id
user_seat_num
flop11
flop12
flop21
flop22
flop31
flop32
turn11
turn12
river11
river12
stage
type2
type3
table_id
button_id
money_i_have
bank_before
pre-flop_player_1_action_type_1
pre-flop_player_1_action_sum_1
pre-flop_player_1_money_had_1
pre-flop_player_1_action_type_2
pre-flop_player_1_action_sum_2
pre-flop_player_1_money_had_2
flop_player_1_action_type_1
flop_player_1_action_sum_1
flop_player_1_money_had_1
flop_player_1_action_type_2
flop_player_1_action_sum_2
flop_player_1_money_had_2
turn_player_1_action_type_1
turn_player_1_action_sum_1
turn_player_1_money_had_1
turn_player_1_action_type_2
turn_player_1_action_sum_2
turn_player_1_money_had_2
river_player_1_action_type_1
river_player_1_action_sum_1
river_player_1_money_had_1
river_player_1_action_type_2
river_player_1_action_sum_2
river_player_1_money_had_2
pre-flop_player_2_action_type_1
pre-flop_player_2_action_sum_1
pre-flop_player_2_money_had_1
pre-flop_player_2_action_type_2
pre-flop_

In [40]:
columns_to_drop = ['unique_id', 'hand_id', 'user_id', 'user_action_sum', 'user_result']

In [41]:
raw_data_actions.drop(columns_to_drop, axis=1, inplace=True)

In [42]:
for col in raw_data_actions.columns:
    print(col)

user_step_num
user_seat_num
flop11
flop12
flop21
flop22
flop31
flop32
turn11
turn12
river11
river12
stage
type2
type3
table_id
button_id
money_i_have
bank_before
pre-flop_player_1_action_type_1
pre-flop_player_1_action_sum_1
pre-flop_player_1_money_had_1
pre-flop_player_1_action_type_2
pre-flop_player_1_action_sum_2
pre-flop_player_1_money_had_2
flop_player_1_action_type_1
flop_player_1_action_sum_1
flop_player_1_money_had_1
flop_player_1_action_type_2
flop_player_1_action_sum_2
flop_player_1_money_had_2
turn_player_1_action_type_1
turn_player_1_action_sum_1
turn_player_1_money_had_1
turn_player_1_action_type_2
turn_player_1_action_sum_2
turn_player_1_money_had_2
river_player_1_action_type_1
river_player_1_action_sum_1
river_player_1_money_had_1
river_player_1_action_type_2
river_player_1_action_sum_2
river_player_1_money_had_2
pre-flop_player_2_action_type_1
pre-flop_player_2_action_sum_1
pre-flop_player_2_money_had_1
pre-flop_player_2_action_type_2
pre-flop_player_2_action_sum_2
pre-

Definining target variable

In [43]:
raw_data_actions = raw_data_actions.loc[raw_data_actions['user_action_type'].isin(['raises', 'bets', 'folds', 'checks', 'calls'])].copy()

In [44]:
raw_data_actions['user_action_type'].unique()

array(['folds', 'calls', 'raises', 'bets', 'checks'], dtype=object)

In [45]:
raw_data_actions['game_continue'] = raw_data_actions['user_action_type'].apply(lambda x: 1 if x in ['raises', 'bets', 'checks', 'calls'] else 0)

In [46]:
raw_data_actions.drop('user_action_type', axis=1, inplace=True)

Preparing for Modeling without embeddings

In [48]:
print(f'Labels are {set(raw_data_actions["game_continue"])}')
ones = raw_data_actions.loc[raw_data_actions['game_continue']==1].shape[0]
zeros = raw_data_actions.loc[raw_data_actions['game_continue']==0].shape[0]
print(f'Number of 1s in a dataset: {ones}, Number of 0s in a dataset: {zeros}')
print(f'Ratio of 1 to total is {ones/(ones+zeros)}')

Labels are {0, 1}
Number of 1s in a dataset: 41699, Number of 0s in a dataset: 36417
Ratio of 1 to total is 0.5338086947616365


Deal with missing

In [49]:
for col in raw_data_actions.columns:
    a = raw_data_actions[col].isna().sum()
    if a > 0:
        print(col, a)
        raw_data_actions[col].fillna('unknown', inplace=True)

flop11 50240
flop12 50240
flop21 50240
flop22 50240
flop31 50240
flop32 50240
turn11 64934
turn12 64934
river11 73120
river12 73120
pre-flop_player_2_action_type_1 2
pre-flop_player_3_action_type_1 16
pre-flop_player_4_action_type_1 174
pre-flop_player_4_action_type_2 6
flop_player_4_action_type_1 31
flop_player_4_action_type_2 5
turn_player_4_action_type_1 16
turn_player_4_action_type_2 5
river_player_4_action_type_1 1


In [50]:
number_of_missing = raw_data_actions.isna().sum()

In [51]:
number_of_missing.sum()

0

In [52]:
def func(x):
    if float(x) == 0.01:
        return 'one'
    elif float(x) == 0.02:
        return 'two'
    elif float(x) == 0.05:
        return 'five'
    elif float(x) == 0.25:
        return 'twentyfive'
    elif float(x) == 0.5:
        return 'fivety'
    elif float(x) == 1.0:
        return 'hundred'
    elif float(x) == 3.0:
        return '3hundred'
    elif float(x) == 0.1:
        return 'zeroone'
    elif float(x) == 2.0:
        return 'twozero'
    elif float(x) == 6.0:
        return 'sixzero'

In [53]:
raw_data_actions['type2'] = raw_data_actions['type2'].apply(func)
raw_data_actions['type3'] = raw_data_actions['type3'].apply(func)

Modeling without embeddings

In [54]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, sum_models


def cv_catboost(X, Y, validation_df, cat_columns, target_column_name, number_of_runs):
    # CROSS VALIDATION
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

    # RUN
    c = 0
    oof_preds = np.zeros((len(X), 2))
    models = []

    for train, valid in cv.split(X, Y):
        print("VAL fold %s" % c)
        X_train = X.iloc[train]
        Y_train = Y.iloc[train]
        X_valid = X.iloc[valid]
        Y_valid = Y.iloc[valid]

        model = CatBoostClassifier(iterations=1500, 
                                   verbose=200,
                                   random_seed=43,
                                   od_type='Iter',
                                   od_wait=200,
                                   train_dir=f'model_cross_val_{c}'
                                   )

        try:
            model.fit(X_train, Y_train,
                    eval_set=(X_valid, Y_valid), 
                    use_best_model=True,
                    cat_features=cat_columns, 
                    save_snapshot=True,
                    snapshot_interval=300,
                    snapshot_file=f'snapshot_{c}',
                    )
        except:
            model.fit(X_train, Y_train,
                    eval_set=(X_valid, Y_valid), 
                    use_best_model=True,
                    cat_features=cat_columns,
                    )
        
        oof_preds[valid] = model.predict_proba(X_valid)
        models.append(model)

        # EVALUATION
        auc = roc_auc_score(validation_df[target_column_name], [r[1] for r in model.predict_proba(validation_df.drop(target_column_name, axis=1))] )
        print(f'AUC for fold {c} is {auc}')
        c += 1
        
        if c == number_of_runs:
            break
        
    final_model = sum_models(models, ctr_merge_policy='LeaveMostDiversifiedTable')
    auc = roc_auc_score(validation_df[target_column_name], [r[1] for r in final_model.predict(validation_df.drop(target_column_name, axis=1), prediction_type='Probability')] )
    print(f'Final model AUC is {auc}')

    # SAVE OOF PREDS
    oof_pred_df = pd.DataFrame(columns=['ID_code', 'target'])
    oof_pred_df['ID_code'] = pd.Series(X.index.tolist())
    oof_pred_df['target'] = pd.Series(oof_preds)

    return final_model, oof_pred_df, models

In [55]:
train_df, validation_df = train_test_split(raw_data_actions, test_size=0.1, stratify=raw_data_actions['game_continue'], random_state=200)

In [56]:
X, Y = train_df.drop(['game_continue'], axis=1), train_df['game_continue']

In [57]:
def get_cat_cols(X, other_method=False):
    categorical_cols = []
    for col in X.columns:
        if other_method is False:
            s = X[col].unique().shape[0]
            if s < 38 and 'sum' not in col and 'money_had' not in col:
                samp = set(X[col].sample(100))
                #print(f'For column {col} the number of unique valriables is {s}, sample {samp}')
                categorical_cols.append(col)
        else:
            if X[col].dtype == object:
                categorical_cols.append(col)
    return categorical_cols

In [58]:
categorical_cols = get_cat_cols(X)

In [59]:
categorical_indices = [X.columns.get_loc(col) for col in categorical_cols]

In [ ]:
import time
start = time.time()
final_model, oof_pred_df, models = cv_catboost(X, Y, validation_df, categorical_indices, 'game_continue', 1)
print(time.time() - start)

VAL fold 0
Learning rate set to 0.071875
Learning rate set to 0.071875
0:	learn: 0.6755014	test: 0.6758373	best: 0.6758373 (0)	total: 526ms	remaining: 13m 7s
200:	learn: 0.4788684	test: 0.4870342	best: 0.4870342 (200)	total: 1m 36s	remaining: 10m 23s
400:	learn: 0.4547244	test: 0.4724106	best: 0.4724106 (400)	total: 4m 12s	remaining: 11m 33s


Get Feature importance

In [600]:
import numpy as np 
from sklearn.metrics import roc_auc_score, accuracy_score


def score(model, X, y):
    y_pred = model.predict(X)
    return accuracy_score(y, y_pred)

def score_auc(model, X, y):
    y_pred_prob = [i[1] for i in model.predict_proba(X)]
    return roc_auc_score(y, y_pred_prob)

def shuffled_scores(X_orig, y, model, score_func):
    scores = []
    for col in X_orig.columns:
        X = X_orig.copy()
        X[col] = np.random.permutation(X[col])
        score = score_func(model, X, y)
        scores.append(score)
    return np.array(scores)

def permutation_feature_importance(X, y, model, score_func, n_iter):
    base_score = score_func(model, X, y)
    print('Base score is ', base_score)
    score_decreases = []
    for i in range(n_iter):
        scores = shuffled_scores(X, y, model, score_func)
        score_decreases.append(-scores + base_score)
    return base_score, score_decreases

def get_permutation_importance(X, y, model, score_func):
    '''
    Call this function to get permutation feature importances.
    X - pandas dataframe with all the features used to train the model
    y - pandas dataframe or series object 
    model - Model object used for prediction. Any model objects with predict or .predict_proba methods are acceptable. 
    score_func - function to score the model. AUC and ACCURACY are available above
    '''
    
    base_score, score_decreases = permutation_feature_importance(X, y, model, score_func, 5)
    permutation_feature_importances = np.mean(score_decreases, axis=0)

    return base_score, permutation_feature_importances

In [475]:
base_score, permutation_feature_importances = get_permutation_importance(validation_df.drop('game_continue', axis=1), validation_df['game_continue'], models[0], score_auc)

Base score is  0.8513074776358369


In [577]:
feat_df = pd.DataFrame()
feat_df['feature'] = validation_df.drop('game_continue', axis=1).columns.tolist()
feat_df['scores'] = permutation_feature_importances
feat_df.sort_values(by=['scores'], ascending=False, inplace=True)

In [578]:
useful_feat_df = feat_df.loc[feat_df['scores']>0.0].copy()

In [579]:
useful_features = useful_feat_df['feature'].tolist()

In [696]:
new_X = X[useful_features].copy()
new_validation_df = validation_df[useful_features + ['game_continue']].copy()
categorical_cols = get_cat_cols(new_X)
categorical_indices = [new_X.columns.get_loc(col) for col in categorical_cols]

KeyError: "None of [Index(['MODE(actions.flop11)', 'SUM(actions.river_player_9_money_had_2)',\n       'SUM(actions.river_player_10_action_sum_1)',\n       'STD(actions.pre-flop_player_4_money_had_1)',\n       'MEAN(actions.pre-flop_player_2_money_had_1 WHERE stage = river)',\n       'MAX(actions.flop_player_2_action_sum_1 WHERE stage = flop)',\n       'MODE(actions.pre-flop_player_3_action_type_1)',\n       'MEAN(actions.river_player_6_action_sum_1)',\n       'MEAN(actions.user_seat_num WHERE stage = pre-flop)',\n       'MAX(actions.flop_player_2_action_sum_2)',\n       'SUM(actions.turn_player_4_action_sum_1)',\n       'MIN(actions.pre-flop_player_3_money_had_1 WHERE stage = river)',\n       'STD(actions.turn_player_1_action_sum_1 WHERE stage = turn)',\n       'STD(actions.turn_player_6_money_had_2)',\n       'MEAN(actions.river_player_1_action_sum_1)',\n       'MAX(actions.pre-flop_player_1_action_sum_2 WHERE stage = turn)',\n       'MAX(actions.pre-flop_player_4_money_had_1 WHERE st

In [697]:
import time
start = time.time()
final_model, oof_pred_df, models = cv_catboost(new_X, Y, new_validation_df, categorical_indices, 'game_continue', 1)
print(time.time() - start)

VAL fold 0


CatBoostError: Invalid cat_features[46] = 102 value: index must be < 102.